<a href="https://colab.research.google.com/github/kudanzakalabo/seminar/blob/main/Pytorch_LSTM_%E6%BC%94%E7%BF%92_%E3%83%AC%E3%83%B3%E3%82%BF%E3%83%AB%E8%87%AA%E8%BB%A2%E8%BB%8A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#レンタル自転車の利用者数予測問題

https://signate.jp/competitions/114

#ライブラリ

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

In [ ]:
from time import time
from datetime import timedelta

# Pytorch ライブラリ

In [ ]:
from torch import nn
from torch.nn import LSTM, Embedding, Linear
from torch.nn.functional import one_hot
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

#データ

In [ ]:
#df_rent_train = pd.read_table("rent_train.tsv")
df_rent_train = pd.read_table('https://tus.box.com/shared/static/bnc2uu2auq466c6zcc31xawmaezdgcmo.tsv')
df_rent_train

In [ ]:
#データ型確認
df_rent_train.info()

In [ ]:
#テストコード - 日付フォーマット
df_rent_train[['dteday','hr']].apply(lambda x: x['dteday'] + ' ' + str(x['hr'])+':00', axis=1)

In [ ]:
#テストコード - 日付フォーマット
pd.to_datetime(df_rent_train[['dteday','hr']].apply(lambda x: x['dteday'] + ' ' + str(x['hr'])+':00', axis=1))

In [ ]:
df_dateindex = df_rent_train.set_index(pd.to_datetime(df_rent_train[['dteday','hr']].apply(lambda x: x['dteday'] + ' ' + str(x['hr'])+':00', axis=1)))
df_dateindex

In [ ]:
df_dateindex.describe()

#基礎分析

In [ ]:
sns.boxplot(data=df_dateindex, x='season',y='cnt')

In [ ]:
sns.boxplot(data=df_dateindex, x='mnth',y='cnt')

In [ ]:
sns.boxplot(data=df_dateindex,x='hr',y='cnt')

In [ ]:
sns.boxplot(data=df_dateindex[df_dateindex['holiday']==1],x='hr',y='cnt')

In [ ]:
sns.boxplot(data=df_dateindex[df_dateindex['holiday']==0],x='hr',y='cnt')

In [ ]:
sns.boxplot(data=df_dateindex,x='weekday',y='cnt')

In [ ]:
sns.boxplot(data=df_dateindex,x='weathersit',y='cnt')

In [ ]:
sns.boxplot(data=df_dateindex[df_dateindex['holiday']==1],x='weathersit',y='cnt')

In [ ]:
sns.boxplot(data=df_dateindex[df_dateindex['holiday']==0],x='weathersit',y='cnt')

In [ ]:
import numpy as np

In [ ]:
df_dateindex.assign(temp=df_dateindex.temp.apply(lambda x: np.round(10*x)))

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data=_[df_dateindex['holiday']==1],x='temp',y='cnt')
plt.show()
plt.figure(figsize=(15,5))
sns.boxplot(data=_[df_dateindex['holiday']==0],x='temp',y='cnt')
plt.show()

In [ ]:
df_dateindex.assign(atemp=df_dateindex.atemp.apply(lambda x: np.round(10*x)))

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data=_[df_dateindex['holiday']==1],x='atemp',y='cnt')
plt.show()
plt.figure(figsize=(15,5))
sns.boxplot(data=_[df_dateindex['holiday']==0],x='atemp',y='cnt')
plt.show()

In [ ]:
df_dateindex.assign(hum=df_dateindex.hum.apply(lambda x: np.round(10*x)))

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data=_[df_dateindex['holiday']==1],x='hum',y='cnt')
plt.show()
plt.figure(figsize=(15,5))
sns.boxplot(data=_[df_dateindex['holiday']==0],x='hum',y='cnt')
plt.show()

In [ ]:
df_dateindex.assign(windspeed=df_dateindex.windspeed.apply(lambda x: np.round(10*x)))

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(data=_[df_dateindex['holiday']==1],x='windspeed',y='cnt')
plt.show()
plt.figure(figsize=(15,5))
sns.boxplot(data=_[df_dateindex['holiday']==0],x='windspeed',y='cnt')
plt.show()

#時系列可視化

In [ ]:
df_dateindex.cnt.plot(figsize=(20,4))

In [ ]:
日毎合計 = df_dateindex.groupby('dteday').sum()

In [ ]:
日毎合計.cnt.plot(figsize=(20,4))

In [ ]:
df_dateindex['cnt'][:1000].plot(figsize=(20,4), marker='1')

In [ ]:
for i,j in df_dateindex.groupby('mnth'):
  j.cnt.plot(figsize=(20,4), marker='+')
  plt.show()

#部分系列データの整形

##スケーリング・正規化

In [ ]:
for i in ['season','mnth','weekday','workingday','weathersit','hr']:
  print(i, df_dateindex[i].value_counts().sort_index().index)

In [ ]:
df_scaled = df_dateindex.assign(cnt = df_dateindex.cnt / df_dateindex.cnt.median()).assign(weathersit = df_dateindex.weathersit -1).assign(season = df_dateindex.weathersit -1).assign(mnth = df_dateindex.mnth -1)

##欠損の補完

In [ ]:
daterange = pd.date_range(start=df_dateindex.index[0], end=df_dateindex.index[-1], freq='H')
daterange

In [ ]:
df_daterange=pd.Series(range(len(daterange)),index=daterange,name='date_id')

In [ ]:
df_nan = pd.merge(df_scaled, df_daterange, left_index=True, right_index=True, how='outer')
df_nan.isna().sum()

In [ ]:
plt.figure(figsize=(20,4))
plt.plot(df_nan.iloc[:743].cnt.fillna(0),marker='1')
plt.show()

##部分系列の分割

In [ ]:
df_nan.cnt = df_nan.cnt.fillna(0)
df_nan = df_nan.ffill()

In [ ]:
#@title 系列長パラメタ
length_pred = 24 #@param
#ターゲット期間
length_target = 6 #@param
#部分系列長=予測変数長+ターゲット期間
length_ss = length_pred + length_target

In [ ]:
#groupby <-> resample (時間インデックスで使用可能)
[j for i,j in df_nan.resample('24h')][0]

In [ ]:
#データフレームの各行をリストでまとめる
df_nan.resample('24h').aggregate(list).head(2)

In [ ]:
df_nan.resample('24h',offset='23h').aggregate(list).head(2)

In [ ]:
pd.concat([df_nan.resample('4h',offset=str(i)+'h').aggregate(list) for i in range(4)]).sort_index()

In [ ]:
df_resample = pd.concat([df_nan.resample(str(length_ss)+'h',offset=str(i)+'h').aggregate(list) 
for i in range(length_ss)]).sort_index()
df_resample.head()

In [ ]:
df_list = df_resample[df_resample.cnt.apply(len) == length_ss]
df_list.head()

##訓練テスト分割：月の初めと終わりに分割

In [ ]:
dteday_meanfill = pd.Series( list(map(lambda x:int(str(x)[8:10]), df_list.index)), index=df_list.index, name='split')
dteday_meanfill

In [ ]:
series_target_train = dteday_meanfill.apply(lambda x: 1 <= int(x) <= 20) 
series_target_test = dteday_meanfill.apply(lambda x: 20 < int(x) )
series_target_test

##静的な変数の生成（季節・気候）

In [ ]:
#名義変数は最頻値をとる
#df_mode = df_list[['season','mnth','weekday','workingday','weathersit','hr']].applymap(lambda x: scipy.stats.mode(x)[0].astype(int)).apply(lambda x:x-x.min(), axis=0)
df_mode = df_list[['season','mnth','weekday','workingday','weathersit','hr']].applymap(lambda x: scipy.stats.mode([i for i in x if not np.isnan(i)])[0].astype(int))
df_mode.describe()

In [ ]:
mode_numpy = np.array(list(map(np.concatenate, df_mode.values)))
mode_numpy

In [ ]:
mode_train = np.array(list(map(np.concatenate, df_mode[series_target_train].values)))
mode_train.shape

In [ ]:
for i in mode_train.T:
  print(np.unique(i))

In [ ]:
mode_test = np.array(list(map(np.concatenate, df_mode[series_target_test].values)))
mode_test.shape

##実数値の変数（気候）

In [ ]:
#数値変数は平均
average_dataframe = df_list[['temp',	'atemp',	'hum',	'windspeed']].applymap(np.nanmean)
average_dataframe

In [ ]:
average_numpy = average_dataframe.values
average_numpy

In [ ]:
average_train = average_dataframe[series_target_train].values
average_train

In [ ]:
average_test = average_dataframe[series_target_test].values
average_test

In [ ]:
#cnt から予測変数と目的変数を生成
sequence_dataframe = pd.merge(df_list.cnt.apply(lambda x:x[:length_pred]).rename('predictors'),
         df_list.cnt.apply(lambda x:np.nanmean(x[-length_target:])).rename('target'),
         left_index=True,right_index=True)
sequence_dataframe

In [ ]:
#部分系列の配列
sequence_numpy = np.array(sequence_dataframe.predictors.values.tolist())[:,:,np.newaxis]
sequence_numpy.shape

In [ ]:
#部分系列の訓練用分割
sequence_train = np.array(sequence_dataframe[series_target_train].predictors.values.tolist())[:,:,np.newaxis]
sequence_train.shape

In [ ]:
#部分系列のテスト用分割
sequence_test = np.array(sequence_dataframe[series_target_test].predictors.values.tolist())[:,:,np.newaxis]
sequence_test.shape

In [ ]:
target_train = np.array(sequence_dataframe[series_target_train].target.values.tolist())[:,np.newaxis]
target_train.shape

In [ ]:
target_test = np.array(sequence_dataframe[series_target_test].target.values.tolist())[:,np.newaxis]
target_test.shape

In [ ]:
print(mode_train[0])
print(mode_train[100])

In [ ]:
list_mode = list(map(lambda x: np.unique(x) - min(x), mode_numpy.T))
list_mode

In [ ]:
sequence_train.shape, sequence_test.shape, average_train.shape, average_test.shape

sequence_train/test が時系列の numpy配列，average_train/test が気候実数値変数のnumpy配列，mode_train/test が名義変数のnumpy 配列

#LSTMモデル

名義変数 (6) > EmbeddingLayer

時系列 (24,1) > LSTM

実数変数 (4): 気象情報 > LSTMの出力・Embeddingの出力と結合

出力： 実数: length_target 時間の平均

In [ ]:
#LSTM : 内部表現の次数
dim_lstm = 10
dim_ts_in = target_train.shape[1]
dim_ts_in

In [ ]:
#名義変数
df_mode.describe()

In [ ]:
#埋め込み層のパラメタ
len_season = 4
len_mnth = 12
len_wd = 7
len_workday = 2
len_weathersit =3
len_hr = 24

In [ ]:
#@title クラス定義
class LSTMRegressor(nn.Module): #Moduleのサブクラス
  def __init__(self):
    super(LSTMRegressor,self).__init__()

    #ハイパーパラメタ
    self.dim_input = length_pred
    self.dim_lstm = dim_lstm
    self.dim_ts_in = dim_ts_in
    self.dim_linear = 10

    #LSTM層
    self.lstm = LSTM(input_size=self.dim_ts_in, hidden_size=self.dim_lstm, batch_first=True)

    self.feature_dims = [2, 2, 2, 2, 2, 2, 4]
    self.use_features = [True, True, True, True, True, True, True]

    #Embedding層
    self.embedding_season = Embedding(len_season, self.feature_dims[0])
    self.embedding_mnth = Embedding(len_mnth, self.feature_dims[1])
    self.embedding_weekday = Embedding(len_wd, self.feature_dims[2])
    self.embedding_workingday = Embedding(len_workday, self.feature_dims[3])
    self.embedding_hr = Embedding(len_hr, self.feature_dims[4])
    self.embedding_weathersit = Embedding(len_weathersit, self.feature_dims[5])

    #各層の出力と気候実変数入力を結合して線形和
    self.linear = torch.nn.Sequential(
        Linear(dim_lstm + 
               sum([i for i,j in zip(self.feature_dims, self.use_features) if j]), 
               self.dim_linear, bias = True), # 10変数出力
        torch.nn.Dropout(),
        Linear(self.dim_linear, 1, bias = True), # 1変数
        torch.nn.ReLU(), # 活性化関数
        )

  def forward(self, X_ts, X_season, X_mnth, X_weekday, X_workingday, X_hr, X_wsit, X_weatherv):
    #LSTM
    x_ts, (h, cell) = self.lstm(X_ts)

    #Embedding
    x_season = self.embedding_season(X_season)
    x_mnth = self.embedding_mnth(X_mnth)
    x_weekday = self.embedding_weekday(X_weekday)
    x_workingday = self.embedding_workingday(X_workingday)
    x_hr = self.embedding_hr(X_hr)
    x_wsit = self.embedding_weathersit(X_wsit)

    #結合
    x = torch.concat([ x_ts[:,-1,:] ] + 
                     [i for i,j in zip(
            [x_season, x_mnth, 
             x_weekday, x_workingday, x_hr, 
             x_wsit, X_weatherv], 
             self.use_features) if j], dim=1)

    #線形結合層
    x = self.linear(x)    

    return x

In [ ]:
#ネットワークオブジェクト初期化
regressor = LSTMRegressor()
regressor

In [ ]:
# forward 関数のテスト
length = 10
for i in range(0, 100, length):

  out = regressor.forward(torch.Tensor(sequence_train[i:i+length]), 
                           torch.LongTensor(mode_train[i:i+length,0]),
                           torch.LongTensor(mode_train[i:i+length,1]),
                           torch.LongTensor(mode_train[i:i+length,2]),
                           torch.LongTensor(mode_train[i:i+length,3]),
                           torch.LongTensor(mode_train[i:i+length,5]),
                           torch.LongTensor(mode_train[i:i+length,4]),
                           torch.FloatTensor(average_train[i:i+length])
                           )
out.shape

In [ ]:
#pip install torchviz

In [ ]:
#from torchviz import make_dot
#make_dot(out, params=dict(regressor.named_parameters()))

#訓練準備

* GPU設定 
* 最適化・学習率
* 損失関数
* 世代数・バッチサイズ
* データローダ

Colabのノートブック設定 > ハードウェアアクセラレータ > GPU > 保存

保存するとセッションが初期化されるので再度実行する

CPUのメモリからGPUのメモリにネットワークパラメタやデータをコピー

In [ ]:
# 学習用のデバイスを設定 GPUは 'cuda'
device = ('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu' #デバッグ時
device

In [ ]:
# GPUのメモリにパラメタをコピー
regressor = regressor.to(device) 

In [ ]:
# 訓練モードを設定（勾配を保存する）
regressor.train()

In [ ]:
# 最適化手法を設定するオブジェクト（学習率）
optimizer = optim.Adam(regressor.parameters(), lr = 0.0005)

In [ ]:
# 損失関数
lossfunc_mse = nn.MSELoss() #二乗誤差損失：スパイクを優先してフィッティング
lossfunc_l1 = nn.L1Loss() # 絶対誤差損失：滑らかに変化する部分が優先されやすい

In [ ]:
# 学習率のスケジュール制御
scheduler = optim.lr_scheduler.MultiStepLR(optimizer,milestones=(),gamma=0.1)

In [ ]:
# 学習のラウンド数
max_epoch = 800
# バッチサイズ
batchsize = 128

In [ ]:
#入力をTensorDatasetとしてまとめる
#ss, season, mnth, weekday, workday, hr, weathersit, weatherv, target
dataset=TensorDataset(torch.Tensor(sequence_train), 
                      torch.LongTensor(mode_train[:,0]),
                      torch.LongTensor(mode_train[:,1]),
                      torch.LongTensor(mode_train[:,2]),
                      torch.LongTensor(mode_train[:,3]),
                      torch.LongTensor(mode_train[:,5]),
                      torch.LongTensor(mode_train[:,4]),
                      torch.FloatTensor(average_train),
                      torch.FloatTensor(target_train)
                      )
len(dataset)

In [ ]:
#ミニバッチを切り出して for 文の中で取り出すためのデータローダ
dataloader = DataLoader(dataset=dataset, shuffle=True, drop_last=False, batch_size=batchsize)
#len はバッチ数
len(dataloader)

#訓練

In [ ]:
#時間を計測(マジックコマンド)
#%%time
#スタート時刻
start = time()
#記録用のリスト
list_loss = []
#記録用の変数
batch_count = 0

#学習の処理を最大epoch数繰り返す
for epoch in range(max_epoch):

  #ログ：残り時間等
  if epoch >0 and epoch % 10 == 0:
    print(epoch, batch_count, epoch_loss, ' ETA: ', (max_epoch-epoch)*timedelta(seconds = time()-start)/(epoch)/60)

  epoch_loss = 0 #epoch毎の損失の合計

  # dataloaderからバッチを取り出す（10回で終了）
  #for i, (ss, season, mnth, weekday, workday, hr, weathersit, weatherv, target) in zip(range(10), dataloader):
  for i, (ss, season, mnth, weekday, workday, hr, weathersit, weatherv, target) in enumerate(dataloader):

    #前に計算した勾配をリセット(0にする)
    optimizer.zero_grad()

    #ネットワークの出力を計算（入力変数はGPUにコピー）
    out = regressor(ss.to(device), season.to(device), mnth.to(device), weekday.to(device), 
                    workday.to(device), hr.to(device), weathersit.to(device), weatherv.to(device))
    
    #損失関数を計算
    #loss = torch.sum(abs(target.to(device) - out))
    loss = lossfunc_l1(target.to(device), out)

    #逆伝播の勾配を計算
    loss.backward()

    #モデルパラメタの更新
    optimizer.step()

    #epoch損失の累積
    epoch_loss += torch.sqrt(loss).item()
    #バッチ数の更新
    batch_count += 1

  #scheduler.step() #学習率のスケジューリング

  #epoch損失の記録
  list_loss.append(epoch_loss)

  #学習曲線
  if epoch > 0 and epoch % 50 ==0:
    #学習曲線プロット
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.plot(list_loss[-50:])
    plt.show()

In [ ]:
#保存用のパスを設定
logpath = '/content/'

In [ ]:
#学習したモデルを保存
torch.save(regressor.state_dict(), logpath+'regressor_'+str(max_epoch)+'.pkl')

#モデル読み込み

In [ ]:
#ファイルエリアにpklファイルを置いてから実行
#<All keys matched successfully>と出力されれば読み込み成功
#regressor.load_state_dict(torch.load(logpath+'regressor_100.pkl'))

#予測

In [ ]:
#テストデータとローダ
testset=TensorDataset(torch.Tensor(sequence_test), 
                      torch.LongTensor(mode_test[:,0]),
                      torch.LongTensor(mode_test[:,1]),
                      torch.LongTensor(mode_test[:,2]),
                      torch.LongTensor(mode_test[:,3]),
                      torch.LongTensor(mode_test[:,5]),
                      torch.LongTensor(mode_test[:,4]),
                      torch.FloatTensor(average_test),
                      torch.FloatTensor(target_test)
                      )

testloader=DataLoader(dataset=testset, shuffle=False, drop_last=False, batch_size=batchsize)

In [ ]:
#評価モード（勾配を計算しない）
regressor.eval()

In [ ]:
dataloader = DataLoader(dataset=dataset, shuffle=False, drop_last=False, batch_size=batchsize)

In [ ]:
#学習済みモデルの訓練データに対する予測を計算
%%time
#40sec
train_pred_list = []
train_loss_sum = 0
batch_count = 0

for i, (ss, season, mnth, weekday, workday, hr, weathersit, weatherv, target) in enumerate(dataloader):

    out = regressor(ss.to(device), season.to(device), mnth.to(device), weekday.to(device), 
                    workday.to(device), hr.to(device), weathersit.to(device), weatherv.to(device))

    loss = lossfunc_l1(target.to(device), out)

    train_pred_list.append(out.tolist())
    train_loss_sum += torch.sqrt(loss).item()
    batch_count += 1

train_pred_array = np.concatenate(train_pred_list)
print(train_pred_array.shape)
print(batch_count)

In [ ]:
#学習済みモデルのテストデータに対する予測を計算

%%time 
#25sec
test_pred_list = []
test_loss_sum = 0
batch_count = 0

for i, (ss, season, mnth, weekday, workday, hr, weathersit, weatherv, target) in enumerate(testloader):
    #出力を計算
    out = regressor(ss.to(device), season.to(device), mnth.to(device), weekday.to(device), 
                    workday.to(device), hr.to(device), weathersit.to(device), weatherv.to(device))
    #損失
    loss = lossfunc_l1(target.to(device), out)

    #出力・損失をリストに保存(tolist,item：値が自動的CPUのメモリにコピーされるので異なるデバイス間のエラーが出難い)
    test_pred_list.append(out.tolist())
    test_loss_sum += torch.sqrt(loss).item()

    batch_count+=1

print(batch_count)

#出力の結合(numpy配列)
test_pred_array = np.concatenate(test_pred_list)

print(test_pred_array.shape)

In [ ]:
#世代数
print(train_loss_sum)
print(test_loss_sum)

#視覚的検証

In [ ]:
df_train_pred = pd.DataFrame({'target':target_train.flatten(),
                           'pred': train_pred_array.flatten()},
                          index=pd.to_datetime(series_target_train.index[series_target_train]))
df_train_pred.plot(alpha=0.3,figsize=(20,4))

In [ ]:
#年月でDFを分割するためdate項目を追加
df_train_pred_ym = df_train_pred.assign(ym = pd.Series(df_train_pred.index).apply(lambda x:str(x)[:7]).values)

In [ ]:
fig, axes = plt.subplots(12,1, figsize=(20,64))

for (i,j), ax in zip(df_train_pred_ym.groupby('ym'), axes.ravel()):
  j.drop('ym',axis=1).plot(title=i, marker='x', ax=ax)

plt.show()

In [ ]:
df_test_pred = pd.DataFrame({'target':target_test.flatten(),
                             'pred':test_pred_array.flatten()},
                            index=series_target_test.index[series_target_test])

df_test_pred.plot(alpha=0.3, figsize=(20,4))

In [ ]:
#年月でDFを分割するためdate項目を追加
df_test_pred_ym = df_test_pred.assign(date = pd.Series(df_test_pred.index).apply(lambda x:str(x)[:7]).values)
df_test_pred_ym

In [ ]:
fig, axes = plt.subplots(6,2, figsize=(16,32))

for (i,j), ax in zip(df_test_pred_ym.groupby('date'), axes.ravel()):
  j.plot(title=i, marker='x', ax=ax)

plt.show()

#学習後の検証事項

予測が単なる後追いになっていないか

損失関数が適切か（絶対誤差・二乗誤差）

時間変動のパターンが再現されているか


#課題
適宜パラメタなどを変更してプログラムを実行し，以下の例のような考察検証する

* テストデータのプロットで検証事項について考察
* 過適合の検証（収束直後のモデルと収束後に学習を続けたモデルでテスト損失を比較）
* 収束に要する epoch 数 (epoch vs 損失のプロットを作成する)
* 学習の収束前と収束後の予測の違い（学習 epoch数の少ないモデルと収束後のモデルでプロットを比較する）
* 損失関数による違い（MSEの場合とL1の場合のプロットを作成して比較する）
* 静的変数を除いた場合の損失と視覚的な検証

#レポート

TeX形式で考察・検証した内容を記述する